In [1]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00


In [2]:
!pip install langchain openai PyPDF2 faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 34.1 MB/s eta 0:00:00


In [3]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 11.9 MB/s eta 0:00:00


In [4]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.1 MB/s eta 0:00:00


In [5]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=788b0d7c90be84986a098cbaf0352b826210555d11e26fb6f94525fa3f0d2f73
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [32]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.0
    Uninstalling openai-1.57.0:
      Successfully uninstalled openai-1.57.0


In [48]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from rouge_score import rouge_scorer
from statistics import mean

In [49]:
# Step 1: Load and Chunk PDF Documents
def load_and_chunk_pdfs(pdf_paths, chunk_size=500, chunk_overlap=50):
    documents = []
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    for pdf_path in pdf_paths:
        loader = PyPDFLoader(pdf_path)
        raw_documents = loader.load()
        documents.extend(text_splitter.split_documents(raw_documents))
    return documents

In [50]:
# Step 2: Create FAISS Vector Store
def create_vector_store(documents, embedding_model="text-embedding-ada-002"):
    embeddings = OpenAIEmbeddings(
        model=embedding_model,
        openai_api_key=os.getenv("OPENAI_API_KEY")  # Ensure the API key is set as an environment variable
    )
    vector_store = FAISS.from_documents(documents, embeddings)
    return vector_store

In [51]:
# Step 3: Create the RAG Pipeline
def create_rag_pipeline(vector_store, llm_model="gpt-4", temperature=0):
    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    qa_chain = RetrievalQA.from_chain_type(
        llm=ChatOpenAI(model=llm_model, temperature=temperature, openai_api_key=os.getenv("OPENAI_API_KEY")),
        retriever=retriever,
        return_source_documents=False
    )
    return qa_chain

In [52]:
# Step 4: Calculate ROUGE Scores
def calculate_rouge(reference, prediction):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, prediction)
    return scores

In [53]:
# Step 5: Query the Pipeline and Calculate Scores
def query_and_evaluate_all(qa_chain, questions, reference_answers):
    """
    Query the RAG model for a list of questions and evaluate them with ROUGE scores.

    Args:
    - qa_chain: The RAG pipeline or QA chain.
    - questions (list): List of questions to query.
    - reference_answers (dict): Dictionary mapping questions to manually generated answers.

    Returns:
    - list: List of dictionaries containing the question, predicted answer, reference answer, and ROUGE scores.
    - dict: Average ROUGE scores across all questions.
    """
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    results = []
    total_rouge_scores = {"rouge1": [], "rouge2": [], "rougeL": []}
    retrieval_metrics = {"precision": [], "recall": [], "f1": []}

    for question in questions:
        try:
            # Query the RAG system for the predicted answer
            predicted_answer = qa_chain.run(question)

            # Retrieve the reference answer
            reference_answer = reference_answers.get(question, None)

            # Evaluate with ROUGE scores
            rouge_scores = None
            if reference_answer:
                rouge_scores = scorer.score(reference_answer, predicted_answer)
                for metric, score in rouge_scores.items():
                    total_rouge_scores[metric].append(score.fmeasure)

            # Store results
            results.append({
                "question": question,
                "predicted_answer": predicted_answer,
                "reference_answer": reference_answer,
                "rouge_scores": {metric: score.fmeasure for metric, score in rouge_scores.items()} if rouge_scores else None
            })

        except Exception as e:
            results.append({
                "question": question,
                "predicted_answer": f"Error during processing: {e}",
                "reference_answer": reference_answers.get(question, None),
                "rouge_scores": None
            })

    # Calculate average ROUGE scores
    average_rouge_scores = {metric: mean(scores) if scores else 0.0 for metric, scores in total_rouge_scores.items()}

    return results, average_rouge_scores

In [54]:
# Step 6: Precision, Recall, and F1 Score for Retrieval
def calculate_retrieval_metrics(qa_chain, questions, relevant_docs):
    """
    Calculate Precision, Recall, and F1 Score for the retrieval component.

    Args:
    - qa_chain: The RAG pipeline or QA chain.
    - questions (list): List of questions to query.
    - relevant_docs (dict): Dictionary mapping questions to lists of relevant document identifiers or content snippets.

    Returns:
    - dict: Dictionary containing Precision, Recall, and F1 Score.
    """
    total_retrieved = 0
    total_relevant = 0
    total_relevant_retrieved = 0

    for question in questions:
        try:
            # Retrieve documents
            retrieved_docs = qa_chain.retriever.get_relevant_documents(question)
            retrieved_set = set([doc.page_content for doc in retrieved_docs])

            # Get relevant documents
            expected_docs = set(relevant_docs.get(question, []))

            # Calculate metrics
            retrieved_relevant = len(retrieved_set.intersection(expected_docs))
            total_retrieved += len(retrieved_set)
            total_relevant += len(expected_docs)
            total_relevant_retrieved += retrieved_relevant

        except Exception as e:
            print(f"Error processing retrieval metrics for question '{question}': {e}")

    precision = total_relevant_retrieved / total_retrieved if total_retrieved else 0
    recall = total_relevant_retrieved / total_relevant if total_relevant else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) else 0

    return {"precision": precision, "recall": recall, "f1": f1_score}

In [57]:
# Main Execution
if __name__ == "__main__":
    # Set OpenAI API Key (ensure it's set as an environment variable for security)
    os.environ["OPENAI_API_KEY"] = "****"

    # Paths to PDF files
    pdf_paths = [
        "Part3_knowledge base.pdf",
        "Part 3_Dataset and Evaluation Question.pdf"
    ]

    # Step 1: Load and Chunk PDF Documents
    documents = load_and_chunk_pdfs(pdf_paths, chunk_size=500, chunk_overlap=50)

    # Step 2: Create FAISS Vector Store
    vector_store = create_vector_store(documents, embedding_model="text-embedding-ada-002")

    # Step 3: Create the RAG Pipeline
    rag_pipeline = create_rag_pipeline(vector_store, llm_model="gpt-4", temperature=0)

    # Step 4: Define Questions and Reference Answers
    questions = [
        "How did Meta’s workforce change by the end of 2023?",
        "What is the report quarter, and when did it end?",
        "What were the key financial highlights this quarter (revenue, gross margin, operating expenses, operating margin, net income, and EPS)?",
        "How much did Meta spend on restructuring for the whole year and Q4?",
        "What happened with Meta’s ad impressions and average price per ad in Q4 and for the whole year?",
        "What’s the revenue outlook for Q1 2024?",
        "What were Meta’s total costs and expenses for Q4 and the full year 2023?",
        "How much cash and marketable securities did Meta have on hand as of December 31, 2023?",
        "What were the main areas Meta invested in during 2023?",
        "How did the Family of Apps and Reality Labs perform in Q4 2023?",
        "How much free cash flow did Meta generate in Q4 and the full year 2023?",
        "Did Meta make any changes to its stock repurchase program or dividends for 2024?",
        "What risks did Meta highlight for 2024?",
        "What drove Meta’s revenue growth in Q4 2023?",
        "How did Reality Labs perform throughout 2023, and what’s Meta’s plan for 2024?"
    ]

    reference_answers = {
        "How did Meta’s workforce change by the end of 2023?": "By the end of 2023, Meta had 67,317 employees, which was a 22.9% decrease compared to 87,314 employees in 2022.",
        "What is the report quarter, and when did it end?": "The reported quarter is the fourth quarter of 2023, which ended on December 31, 2023.",
        "What were the key financial highlights this quarter (revenue, gross margin, operating expenses, operating margin, net income, and EPS)?": "Revenue: $40.11 billion, Gross Margin: 41%, Operating Expenses: $23.73 billion, Operating Margin: 41%, Net Income: $14.02 billion, EPS: $5.33.",
        "How much did Meta spend on restructuring for the whole year and Q4?": "Meta spent $3.45 billion on restructuring for the full year 2023 and $1.15 billion for the fourth quarter.",
        "What happened with Meta’s ad impressions and average price per ad in Q4 and for the whole year?": "In Q4 2023, ad impressions increased by 21% year-over-year, and the average price per ad increased by 2%. For the full year 2023, ad impressions increased by 28% year-over-year, while the average price per ad decreased by 9%.",
        "What’s the revenue outlook for Q1 2024?": "The revenue outlook for Q1 2024 is expected to be in the range of $34.5–37 billion.",
        "What were Meta’s total costs and expenses for Q4 and the full year 2023?": "Total costs and expenses were $23.73 billion for Q4 2023 and $88.15 billion for the full year 2023.",
        "How much cash and marketable securities did Meta have on hand as of December 31, 2023?": "Meta had $65.40 billion in cash, cash equivalents, and marketable securities as of December 31, 2023.",
        "What were the main areas Meta invested in during 2023?": "Meta primarily invested in data center infrastructure, artificial intelligence hardware, and augmented/virtual reality product development.",
        "How did the Family of Apps and Reality Labs perform in Q4 2023?": "Family of Apps Revenue: $39.04 billion, Reality Labs Revenue: $1.07 billion, Reality Labs recorded a loss of $4.65 billion during Q4 2023.",
        "How much free cash flow did Meta generate in Q4 and the full year 2023?": "Meta generated $11.50 billion in free cash flow during Q4 2023 and $43.01 billion for the full year.",
        "Did Meta make any changes to its stock repurchase program or dividends for 2024?": "Yes, Meta initiated a quarterly dividend of $0.50 per share and announced a $50 billion increase in its share repurchase authorization.",
        "What risks did Meta highlight for 2024?": "Meta highlighted risks including regulatory challenges in the EU and U.S., evolving AI infrastructure demands, and operating losses in Reality Labs due to ongoing investments.",
        "What drove Meta’s revenue growth in Q4 2023?": "Revenue growth in Q4 2023 was driven by increased ad impressions and improved advertising performance across the Family of Apps.",
        "How did Reality Labs perform throughout 2023, and what’s Meta’s plan for 2024?": "Reality Labs recorded a total loss of $16.12 billion in 2023. For 2024, Meta plans to continue investments in augmented/virtual reality product development and scaling its ecosystem."
    }

    # Step 5: Query and Evaluate
    results, average_rouge_scores = query_and_evaluate_all(rag_pipeline, questions, reference_answers)

    # Step 6: Print Final Results (Q1 to Q15)
    for idx, result in enumerate(results, 1):
        print(f"Q{idx}: {result['question']}")
        print(f"Predicted Answer: {result['predicted_answer']}")
        print(f"Reference Answer: {result['reference_answer']}")
        if result["rouge_scores"]:
            print("ROUGE Scores:")
            for metric, score in result["rouge_scores"].items():
                print(f"{metric}: {score:.4f}")
        print("\n" + "-" * 50)

    # Step 7: Calculate and Print Average ROUGE Scores
    print("\nAverage ROUGE Scores:")
    for metric, avg_score in average_rouge_scores.items():
        print(f"{metric}: {avg_score:.4f}")

Q1: How did Meta’s workforce change by the end of 2023?
Predicted Answer: By the end of 2023, Meta had 67,317 employees. In 2022, Meta had 87,314 employees, so Meta had a 22.9% decrease in 2023 compared to 2022.
Reference Answer: By the end of 2023, Meta had 67,317 employees, which was a 22.9% decrease compared to 87,314 employees in 2022.
ROUGE Scores:
rouge1: 0.8077
rouge2: 0.6800
rougeL: 0.6538

--------------------------------------------------
Q2: What is the report quarter, and when did it end?
Predicted Answer: The reported quarter is the fourth quarter of 2023. The quarter ended on December 31, 2023.
Reference Answer: The reported quarter is the fourth quarter of 2023, which ended on December 31, 2023.
ROUGE Scores:
rouge1: 0.9032
rouge2: 0.8276
rougeL: 0.9032

--------------------------------------------------
Q3: What were the key financial highlights this quarter (revenue, gross margin, operating expenses, operating margin, net income, and EPS)?
Predicted Answer: The key fin

In [79]:
# Step 9: Fine-Tuning the RAG Pipeline
print("\n🔄 Fine-tuning the RAG pipeline...")

# Fine-Tuning Adjustments:
# - Adjust chunk size and overlap
# - Increase number of retrieved documents (k)
# - Optionally switch embedding models

# Example: Adjusting chunk size and overlap
fine_tuned_documents = load_and_chunk_pdfs(pdf_paths, chunk_size=200, chunk_overlap=100)
print(f"Fine-tuned total chunks created: {len(fine_tuned_documents)}")

# Example: Creating a fine-tuned vector store
fine_tuned_vector_store = create_vector_store(fine_tuned_documents, embedding_model="text-embedding-ada-002")
print("Fine-tuned vector store created.")

# Example: Adjusting number of retrieved documents (k)
fine_tuned_retriever = fine_tuned_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 7})

# Example: Creating a fine-tuned RAG pipeline
fine_tuned_rag_pipeline = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=OPENAI_API_KEY),
    retriever=fine_tuned_retriever,
    return_source_documents=False
)
print("Fine-tuned RAG pipeline created.")


🔄 Fine-tuning the RAG pipeline...
Fine-tuned total chunks created: 233
Fine-tuned vector store created.
Fine-tuned RAG pipeline created.


In [80]:
# Step 10: Re-Evaluate with Fine-Tuned Pipeline
print("\nRe-evaluating with the fine-tuned RAG pipeline...")
fine_tuned_results, fine_tuned_average_rouge_scores = query_and_evaluate_all(fine_tuned_rag_pipeline, questions, reference_answers)
print("Fine-tuned evaluation complete.")


Re-evaluating with the fine-tuned RAG pipeline...
Fine-tuned evaluation complete.


In [82]:
# Step 11: Print Fine-Tuned Results (Q1 to Q15) and Average Scores
print("\n🔵 Fine-Tuned RAG Pipeline Results:")
for idx, result in enumerate(fine_tuned_results, 1):
    print(f"\nQ{idx}: {result['question']}")
    print(f"Predicted Answer: {result['predicted_answer']}")
    print(f"Reference Answer: {result['reference_answer']}")
    if result["rouge_scores"]:
        print("ROUGE Scores:")
        for metric, score in result["rouge_scores"].items():
            print(f"{metric}: {score:.4f}")
    print("-" * 50)

# Print Average ROUGE Scores
print("\n🔵 Average ROUGE Scores:")
for metric, avg_score in fine_tuned_average_rouge_scores.items():
    print(f"{metric}: {avg_score:.4f}")


🔵 Fine-Tuned RAG Pipeline Results:

Q1: How did Meta’s workforce change by the end of 2023?
Predicted Answer: By the end of 2023, Meta had 67,317 employees. In 2022, Meta had 87,314 employees, so Meta had a 22.9% decrease in 2023 compared to 2022.
Reference Answer: By the end of 2023, Meta had 67,317 employees, which was a 22.9% decrease compared to 87,314 employees in 2022.
ROUGE Scores:
rouge1: 0.8077
rouge2: 0.6800
rougeL: 0.6538
--------------------------------------------------

Q2: What is the report quarter, and when did it end?
Predicted Answer: The reported quarter is the fourth quarter of 2023. The quarter ended on December 31, 2023.
Reference Answer: The reported quarter is the fourth quarter of 2023, which ended on December 31, 2023.
ROUGE Scores:
rouge1: 0.9032
rouge2: 0.8276
rougeL: 0.9032
--------------------------------------------------

Q3: What were the key financial highlights this quarter (revenue, gross margin, operating expenses, operating margin, net income, an

In [84]:
import pandas as pd

# Create a DataFrame for ROUGE Score Comparison
rouge_metrics = list(average_rouge_scores.keys())

# Ensure consistent ordering of metrics
rouge_metrics.sort()

# Extract scores
initial_scores = [average_rouge_scores.get(metric, 0.0) for metric in rouge_metrics]
fine_tuned_scores = [fine_tuned_average_rouge_scores.get(metric, 0.0) for metric in rouge_metrics]

# Construct the DataFrame
df_rouge = pd.DataFrame({
    'ROUGE Metric': rouge_metrics,
    'Initial': initial_scores,
    'Fine-Tuned': fine_tuned_scores
})

# Set ROUGE Metric as the DataFrame index for better readability
df_rouge.set_index('ROUGE Metric', inplace=True)

# Display the DataFrame
print("\n🔵 Average ROUGE Scores Comparison:")
display(df_rouge)


🔵 Average ROUGE Scores Comparison:


,Initial,Fine-Tuned
ROUGE Metric,,
rouge1,0.554198,0.564558
rouge2,0.422632,0.432833
rougeL,0.520382,0.494851
